<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/chooseroption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for formatting rates/vols if needed (from previous examples)
def format_rate(r):
    return f"{r * 100:.4f}%"
def format_vol(v):
    return f"{v * 100:.4f}%"

class ChooserOptionTests(unittest.TestCase):

    def setUp(self):
        """Set up the calculation date before each test."""
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Use a fixed date for consistency, or use today's date and set it
        self.today = ql.Date(15, ql.May, 2024) # Example fixed date
        ql.Settings.instance().evaluationDate = self.today

    def tearDown(self):
        """Restore the calculation date after each test."""
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def testAnalyticSimpleChooserEngine(self):
        """Testing analytic simple chooser option."""
        print("Testing analytic simple chooser option...")

        # Market data setup mirroring C++ test
        dc = ql.Actual360()
        today = ql.Settings.instance().evaluationDate # Use the date set in setUp

        spot_q = ql.SimpleQuote(50.0)
        qRate_q = ql.SimpleQuote(0.0)
        rRate_q = ql.SimpleQuote(0.08)
        vol_q = ql.SimpleQuote(0.25)

        spot_h = ql.QuoteHandle(spot_q)
        qTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(qRate_q), dc))
        rTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(rRate_q), dc))
        volTS_h = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.TARGET(), ql.QuoteHandle(vol_q), dc)) # Assuming TARGET calendar like previous tests if not specified

        stochProcess = ql.BlackScholesMertonProcess(spot_h, qTS_h, rTS_h, volTS_h)
        engine = ql.AnalyticSimpleChooserEngine(stochProcess)

        # Option details
        strike = 50.0
        exerciseDate = today + ql.Period(180, ql.Days)
        exercise = ql.EuropeanExercise(exerciseDate)
        choosingDate = today + ql.Period(90, ql.Days)

        option = ql.SimpleChooserOption(choosingDate, strike, exercise)
        option.setPricingEngine(engine)

        # Verification
        calculated = option.NPV()
        expected = 6.1071
        tolerance = 3e-5

        self.assertAlmostEqual(calculated, expected, delta=tolerance,
                               msg=(f"Simple Chooser Option NPV mismatch:\n"
                                    f"  Calculated: {calculated:.6f}\n"
                                    f"  Expected:   {expected:.6f}\n"
                                    f"  Tolerance:  {tolerance}"))


    def testAnalyticComplexChooserEngine(self):
        """Testing analytic complex chooser option."""
        print("Testing analytic complex chooser option...")

        # Market data setup
        dc = ql.Actual360()
        today = ql.Settings.instance().evaluationDate # Use the date set in setUp

        spot_q = ql.SimpleQuote(50.0)
        qRate_q = ql.SimpleQuote(0.05)
        rRate_q = ql.SimpleQuote(0.10)
        vol_q = ql.SimpleQuote(0.35)

        spot_h = ql.QuoteHandle(spot_q)
        qTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(qRate_q), dc))
        rTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(rRate_q), dc))
        volTS_h = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.TARGET(), ql.QuoteHandle(vol_q), dc))

        stochProcess = ql.BlackScholesMertonProcess(spot_h, qTS_h, rTS_h, volTS_h)
        engine = ql.AnalyticComplexChooserEngine(stochProcess)

        # Option details
        callStrike = 55.0
        putStrike = 48.0
        choosingDate = today + ql.Period(90, ql.Days)
        callExerciseDate = choosingDate + ql.Period(180, ql.Days)
        putExerciseDate = choosingDate + ql.Period(210, ql.Days)

        callExercise = ql.EuropeanExercise(callExerciseDate)
        putExercise = ql.EuropeanExercise(putExerciseDate)

        option = ql.ComplexChooserOption(choosingDate, callStrike, putStrike, callExercise, putExercise)
        option.setPricingEngine(engine)

        # Verification
        calculated = option.NPV()
        expected = 6.0508
        tolerance = 1e-4
        error = abs(calculated - expected)

        self.assertLessEqual(error, tolerance,
                             msg=(f"Failed to reproduce complex chooser option value:\n"
                                  f"  Expected:   {expected:.6f}\n"
                                  f"  Calculated: {calculated:.6f}\n"
                                  f"  Error:      {error:.6f}\n"
                                  f"  Tolerance:  {tolerance}"))

if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)